In [1]:
from scipy.optimize import linprog
import pulp
import numpy as np 

In [2]:
#例24.1

In [3]:
c=[2,3]
A=[[1,2],[4,0],[0,4]]
b=[8,16,12]
#新建一个线性规划问题，LpMaximize指明为求最大值
m=pulp.LpProblem(sense=pulp.LpMaximize)
#添加变量
x=[pulp.LpVariable(f'x{i}',lowBound=0) for i in [1,2]]
#目标函数
m+=pulp.lpDot(c,x)
#添加约束
for i in range(len(A)):
    m+=(pulp.lpDot(A[i],x)<=b[i])
#求解
m.solve()
#显示目标函数及对应变量值
pulp.value(m.objective),[pulp.value(var) for var in x]

(14.0, [4.0, 2.0])

In [4]:
#例24.2

In [5]:
#使用linprog()函数求运输问题
def transport_linprog(costs,supply,demand):
    m=len(supply)
    n=len(demand)
    A_ub=np.zeros((m,m*n))
    for row in np.arange(m):
        for col in np.arange(row*n,row*n+n):
            A_ub[row][col]=1
    A_eq=np.zeros((n,m*n))
    for row in np.arange(n):
        for col in np.arange(m):
            A_eq[row][row+n*col]=1
    c=np.ravel(costs)
    result=linprog(c,A_ub=A_ub,b_ub=supply,A_eq=A_eq,\
                   b_eq=demand,method='simplex')
    #以字典的形式返回结果
    return {'success':result.success,'fun':result.fun.round(3),\
            'x':result.x.round(3).reshape((m,n))}

In [6]:
c=[[3,11,3,10],[1,9,2,8],[7,4,10,5]]
supply=[7,4,9]
demand=[3,6,5,6]
result=transport_linprog(c,supply,demand)
#显示结果
print(result['success'])
print(result['fun'])
print(result['x'])

True
85.0
[[0. 0. 5. 2.]
 [3. 0. 0. 1.]
 [0. 6. 0. 3.]]


In [7]:
#例24.3

In [8]:
#使用pulp求解运输问题
def transport_pulp(costs,supply,demand,cat=pulp.LpContinuous):
    #提供给使用人的函数基本信息描述
    """
    ============================================================
    运筹学之运输问题：
    参数：
    costs:运输单价表
    supply:（各点）可供应量
    demand:（各点）需求量
    cat=pulp.LpContinuous(default),pulp.LpInteger or pulp.Binary
    ============================================================
    返回:
    {'fun'：fun ,'x'：x }
    ============================================================
    """
    rows=len(costs)
    cols=len(costs[0])
    problem=pulp.LpProblem('trans_p',sense=pulp.LpMinimize)
    var=[[pulp.LpVariable(f'x{i}{j}',lowBound=0,cat=cat) for j in \
        range(cols)] for i in range(rows)]
    problem += sum([pulp.lpDot(costs[row],var[row]) for row in \
        range(rows)])
    for row in range(rows):
        problem += (pulp.lpSum(var[row]) <= supply[row])
    for col in range(cols):
        problem += (pulp.lpSum([var[row][col] for row in range(rows)])\
             == demand[col])
    problem.solve()
    fun=pulp.value(problem.objective)
    x=[[pulp.value(var[row][col]) for col in range(cols)] for row in \
        range(rows)]
    return {'fun':fun,'x':x}

In [9]:
transport_pulp?

Signature: transport_pulp(costs, supply, demand, cat='Continuous')
Docstring:
运筹学之运输问题：
参数：
costs:运输单价表
supply:（各点）可供应量
demand:（各点）需求量
cat=pulp.LpContinuous(default),pulp.LpInteger or pulp.Binary
返回:
{'fun'：fun ,'x'：x }
File:      e:\高数、统计、运筹文稿及代码\代码\运筹代码\<ipython-input-8-7ca2eafe8b66>
Type:      function


In [10]:
c=[[3,11,3,10],[1,9,2,8],[7,4,10,5]]
supply=[7,4,9]
demand=[3,6,5,6]
result=transport_pulp(c,supply,demand)
print(result['fun'])
print(np.array(result['x']))

85.0
[[2. 0. 5. 0.]
 [1. 0. 0. 3.]
 [0. 6. 0. 3.]]


In [11]:
#例24.4

In [12]:
M=10000
c=[[16,16,13,22,17,17],[14,14,13,19,15,15],[19,19,20,23,M,M],\
    [M,0,M,0,M,0]]
supply=[50,60,50,50]
demand=[30,20,70,30,10,50]
result_1=transport_linprog(c,supply,demand)
print('By trandport_linprog:\nfun:{}\nx:{}'.format(result_1['fun'],\
    np.array(result_1['x'])))
result_2=transport_pulp(c,supply,demand)
print('\n\nBy trandport_pulp:\nfun:{}\nx:{}'.format(result_2['fun'],\
    np.array(result_2['x'])))

By trandport_linprog:
fun:2460.0
x:[[ 0.  0. 50.  0.  0.  0.]
 [ 0.  0. 20.  0. 10. 30.]
 [30. 20.  0.  0.  0.  0.]
 [ 0.  0.  0. 30.  0. 20.]]


By trandport_pulp:
fun:2460.0
x:[[ 0.  0. 50.  0.  0.  0.]
 [ 0.  0. 20.  0. 10. 30.]
 [30. 20.  0.  0.  0.  0.]
 [ 0.  0.  0. 30.  0. 20.]]


In [13]:
#例24.5

In [14]:
M=10000
c=[[10.8,10.95,11.1,11.25],[M,11.1,11.25,11.4],\
    [M,M,11.0,11.15],[M,M,M,11.3]]
supply=[25,35,30,10]
demand=[10,15,25,20]
result=transport_pulp(c,supply,demand,\
    cat=pulp.LpInteger)
result['fun'],np.array(result['x'])

(773.0, array([[10., 10.,  0.,  5.],
        [ 0.,  5.,  0.,  0.],
        [ 0.,  0., 25.,  5.],
        [ 0.,  0.,  0., 10.]]))

In [18]:
from time import time
start=time()
np.random.RandomState(0)
supply_num=30
demand_num=45
c=np.random.randint(0,10,size=(supply_num,demand_num))
supply=np.random.randint(20000,25000,size=supply_num)
demand=np.random.randint(10000,20000,size=demand_num)
result=transport_linprog(c,supply,demand)
#因为随机性及主机运行速度原因，显示的结果会有差别
print((time()-start))
print(result['success'])

0.9484517574310303
True
